In [1]:
from Models.BaselineLSTMWithResourceValidTraceCf import BaselineLSTMWithResourceValidTraceCf
from Utils.Preprocessing import dataset_split
from Models import BaseNN, BaselineLSTMWithResource
from Data.MedicalDataset import MedicalDataset
from typing import List, Tuple
from matplotlib import pyplot as plt

import pandas as pd
from Models.BaselineLSTM import BaselineLSTM
from CustomExceptions.Exceptions import NotSupportedError
from Parameters.EnviromentParameters import EnviromentParameters
from Parameters.Enums import SelectableDatasets, SelectableLoss, SelectableModels, SelectableOptimizer
from Parameters import TrainingParameters
from Utils.PrintUtils import print_big, print_peforming_task
from Data import BPI2012ValidTraceDataset, XESDataset, XESDatasetWithResource
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sn
import tensorflow as tf
import numpy as np
import pathlib
from matplotlib.lines import Line2D
import os
from Utils.SaveUtils import save_parameters_json

In [2]:
from Controller import TraceClassifierController
from Utils.SaveUtils import load_parameters
from Parameters import TrainingParameters
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
### Init dataset 
dataset = BPI2012ValidTraceDataset(
file_path=EnviromentParameters.BPI2012ValidTraceDataset.file_path,
preprocessed_folder_path=EnviromentParameters.BPI2012ValidTraceDataset.preprocessed_foldr_path,
preprocessed_df_type=EnviromentParameters.BPI2012ValidTraceDataset.preprocessed_df_type)



| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012_valid_trace/All 


In [4]:
parameters = TrainingParameters()
tf.random.set_seed(parameters.dataset_split_seed)
np.random.seed(parameters.dataset_split_seed)

In [5]:
train_dataset, test_dataset, validation_dataset = dataset_split(list(range(len(
            dataset))), parameters.train_test_split_portion, seed=parameters.dataset_split_seed,  shuffle=True)

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    train_dataset).batch(parameters.batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices(
    test_dataset).batch(parameters.batch_size)
validation_dataset = tf.data.Dataset.from_tensor_slices(
    validation_dataset).batch(parameters.batch_size)

In [7]:
optim = tf.keras.optimizers.Adam(
                learning_rate=parameters.optimizerParameters.learning_rate)

In [8]:
parameters.optimizerParameters.learning_rate

0.05

In [9]:
batch_index = (list(train_dataset.as_numpy_iterator()))[0]

In [10]:
caseids, padded_data_traces, lengths, padded_data_resources, batch_amount, padded_target_traces, real_trace = dataset.collate_fn(batch_index)

In [11]:
caseids[10]

'197674'

In [12]:
lengths[10]

18

In [13]:
real_trace[10, :]

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int32)

In [14]:
padded_data_traces[10]

array([ 2, 12,  9, 10, 22,  3,  8, 17, 15, 19, 22, 18, 24, 13,  5,  4, 11,
       25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [15]:
padded_target_traces[10]

array([12,  9, 10, 22,  3,  8, 17, 15, 19, 22, 18, 24, 13,  5,  4, 11, 25,
        1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [16]:
model = BaselineLSTMWithResourceValidTraceCf(
            activity_vocab=dataset.activity_vocab,
            resource_vocab=dataset.resource_vocab,
            dense_dim=parameters.baselineLSTMWithResourceparameters.dense_dim,
            activity_embedding_dim=parameters.baselineLSTMWithResourceparameters.activity_embedding_dim,
            resource_embedding_dim=parameters.baselineLSTMWithResourceparameters.resource_embedding_dim,
            lstm_hidden=parameters.baselineLSTMWithResourceparameters.lstm_hidden,
            dropout=parameters.baselineLSTMWithResourceparameters.dropout,
        )


another_model = BaselineLSTMWithResource(
                vocab=dataset.activity_vocab,
                resources=dataset.resource_vocab.vocabs,
                dense_dim=parameters.baselineLSTMWithResourceparameters.dense_dim,
                activity_embedding_dim=parameters.baselineLSTMWithResourceparameters.activity_embedding_dim,
                resource_embedding_dim=parameters.baselineLSTMWithResourceparameters.resource_embedding_dim,
                lstm_hidden=parameters.baselineLSTMWithResourceparameters.lstm_hidden,
                dropout=parameters.baselineLSTMWithResourceparameters.dropout,
            )


In [17]:
with tf.GradientTape() as tape:
    out = another_model(padded_data_traces,
        padded_data_resources, batch_amount, training=True)
    

In [18]:
grads = tape.gradient(out, another_model.trainable_variables)

In [24]:
with tf.GradientTape() as tape:
    out = model(padded_data_traces,
        padded_data_resources, batch_amount, training=True)
    # loss_all = tf.keras.losses.binary_crossentropy(
    #             y_true=real_trace[:, :, tf.newaxis], y_pred=out)
    # mask = tf.cast(real_trace != -1, dtype=tf.float32)
    # loss_all = loss_all * mask
    # loss = tf.reduce_mean(loss_all)

Is training: True


In [25]:
grads = tape.gradient(out, model.trainable_variables)

In [26]:
grads

9781e-02, -4.70926476e+00, -4.21057320e+00,
         1.41773164e-01,  1.64061337e+01, -8.23366642e-02,  5.83772993e+00,
         2.04265296e-01,  4.77095699e+00, -1.01678276e+00, -1.37558479e+01,
         9.01363277e+00,  6.75127506e-02,  2.63117361e+00,  2.24038911e+00,
        -7.67141533e+00, -3.95807505e+00, -8.46972644e-01,  1.81396091e+00,
        -6.11206627e+00,  1.44243270e-01, -6.32800162e-03,  1.26010799e+01,
         5.28533888e+00,  1.16019478e+01, -4.26427454e-01,  6.42656326e+00,
        -2.63656783e+00,  5.34275961e+00, -2.43960357e+00,  9.88672972e-01,
        -1.35464668e+01,  4.41136265e+00, -1.05867493e+00, -1.95632839e+00,
         7.91166830e+00,  5.13856697e+00, -1.05847597e+00,  3.57112706e-01,
         4.49872553e-01, -2.64809728e-01,  5.98645496e+00,  6.97880888e+00,
        -9.55908895e-02, -1.34960899e+01, -8.63423884e-01,  5.31983805e+00,
         1.71527696e+00, -1.63983554e-01, -2.54628038e+00,  2.00852193e-02,
        -3.06336552e-01, -4.09857154e-01,  2

<tf.Tensor: shape=(64, 96), dtype=float32, numpy=
array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32)>